## Preprocessing

## Data load

In [1]:
# 사용할 GPU 번호 선택
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3,5,6"

## Load image

In [2]:
import pandas as pd
# MSI Images
msi_path = '/home/neal1202/MSI_prediction/mss_vs_msi/msi/' 
msi_list = []
for (path, dir, files) in os.walk(msi_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            msi_list.append(filePath)  

# Create Data Frame
# MSI = 0
msi_df = pd.DataFrame(data={'filename': msi_list, 
                               'label': [0 for i in range(len(msi_list))]})

len(msi_list), len(msi_df)

(5260, 5260)

In [3]:
# MSS Images
mss_path = '/home/neal1202/MSI_prediction/mss_vs_msi/mss/' 
mss_list = []
for (path, dir, files) in os.walk(mss_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.jpg':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            mss_list.append(filePath)  

# Create Data Frame
# MSS = 1
mss_df = pd.DataFrame(data={'filename': mss_list, 
                               'label': [1 for i in range(len(mss_list))]})

len(mss_list), len(mss_df)

(5271, 5271)

In [4]:
# Merge MSI + MSS ImageList
train_img = msi_list + mss_list
img_df = mss_df.append(msi_df)

# Shuffle
sh_df = img_df.sample(frac=1).reset_index(drop=True)

len(train_img), len(img_df), len(sh_df)

(10531, 10531, 10531)

In [5]:
# Matplotlib을 이용하여 Image 읽기 (Wait)
from matplotlib.image import imread

im_list = []

for path in sh_df['filename']:
    img = imread(path)
    im_list.append(img)

In [6]:
import numpy as np

x = np.array(im_list)
y = np.array(sh_df['label'])

### [3] Image resize and data set split

In [7]:
from PIL import Image

# Resize Image
train = []

for img in x:
    arr = Image.fromarray(img)
    res = arr.resize((299, 299))
    train.append(np.array(res))
    
train = np.array(train)

In [8]:
from sklearn.model_selection import train_test_split

# Split Data
x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.2)

In [9]:
# Generalize (rescale [0,255] --> [0,1])
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [10]:
print('training dataset =', len(x_train))
print('training label =', len(y_train))

print('test dataset =', len(x_test))
print('test label =', len(y_test))

training dataset = 8424
training label = 8424
test dataset = 2107
test label = 2107


In [11]:
import tensorflow as tf
print(tf.__version__)

/home/neal1202/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

2.0.0-beta1


/home/neal1202/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/neal1202/.local/lib/python3.6/site-packages/tensorboard/co

## Call a Inception pretrained model

In [12]:
import tensorflow as tf

In [13]:
from keras.applications import InceptionV3
conv_base = InceptionV3(weights='imagenet', 
                              include_top=False, 
                              input_shape=(299, 299, 3))

In [17]:
# from keras.applications import InceptionResNetV2
# conv_base = InceptionResNetV2(weights='imagenet',
#                               include_top=False,
#                               input_shape=(299, 299, 3))

In [18]:
import keras
from keras import layers
from keras import models
from keras import regularizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu', 
                      kernel_regularizer=regularizers.l2(0.01))),
model.add(layers.Dense(1, activation='sigmoid'))

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 8, 8, 1536)        54336736  
_________________________________________________________________
flatten_1 (Flatten)          (None, 98304)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               25166080  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 79,503,073
Trainable params: 79,442,529
Non-trainable params: 60,544
_________________________________________________________________


In [20]:
print('Number of trainable weights before freezing the conv base:', len(model.non_trainable_weights))
conv_base.trainable = True
print('Number of trainable weights after freezing the conv base:', len(model.trainable_weights))

Number of trainable weights before freezing the conv base: 408
Number of trainable weights after freezing the conv base: 492


In [21]:
from keras import optimizers
model.compile(loss='binary_crossentropy', 
             optimizer=optimizers.RMSprop(lr=2e-5), metrics=['acc']) # 0.00002

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
from livelossplot.keras import PlotLossesCallback
plot_losses_call_back = PlotLossesCallback()

hist = model.fit(x_train, y_train, epochs=30, batch_size=30, 
                 validation_split=0.2, verbose=0,                 
                 callbacks=[plot_losses_call_back])

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[1088,384,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node inception_resnet_v2/block17_13_conv/convolution (defined at /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[inception_resnet_v2/batch_normalization_193/cond/else/_3649/batchnorm/mul-1-0-TransposeNCHWToNHWC-LayoutOptimizer/_8020]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[1088,384,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node inception_resnet_v2/block17

In [ ]:
import matplotlib.pyplot as plt # Plotting & Image-displaying
%matplotlib inline 

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

plt.title('Training Process', fontsize=15)

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch', fontsize=11)
loss_ax.set_ylabel('loss', fontsize=11)
acc_ax.set_ylabel('accuray', fontsize=11)

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

fig = plt.gcf()
plt.show()
#fig.savefig('Class_InceptionV3_20190803.png')

In [ ]:
# Test Data Set의 Normal/Cancer 비율 확인하기
MSI = 0
MSS = 1

for i in y_test:
    if i == 1.0:
        MSI += 1
    else:
        MSS += 1
        
print('MSI:', MSI, ', MSS:', MSS)

In [ ]:
# 테스트 데이터로 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
from sklearn.metrics import roc_auc_score, auc, roc_curve

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()

y_score = model.predict_proba(x_test)
n_classes = y_test.shape[0]

In [ ]:
y_test

In [ ]:
y_score

In [ ]:
n_classes

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

lw = 2
class_F = y_test
proba_F = y_score

false_positive_rate_F, true_positive_rate_F, thresholds_F = roc_curve(class_F, proba_F)
roc_auc_F = auc(false_positive_rate_F, true_positive_rate_F)

plt.title('MSI Classification AUROC\n(trained by TCIA)\n(Internal Validation with TICA)', fontsize=15)
plt.xlabel('False Positive Rate(1 - Specificity)', fontsize=13)
plt.ylabel('True Positive Rate(Sensitivity)', fontsize=13)

plt.plot(false_positive_rate_F, true_positive_rate_F, 'darkorange', 
         label='ROC curve (AUC = %0.5f)'% roc_auc_F)

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', label='Random guess')

plt.legend(loc='lower right')
plt.show()
#fig.savefig('Cancer_AUROC_InceptionV3_E50_TCGA_20191118.png')

In [ ]:
os.chdir('/home/neal1202/MSI_prediction/model/')

In [ ]:
os.listdir()

In [ ]:
# 모델 저장하기
from keras.models import load_model
model.save('InceptionV3_epoch30_0819.h5')